In [1]:
import nltk
import string
from nltk.corpus import gutenberg
from neo4j.v1 import GraphDatabase, basic_auth
from nltk.stem import WordNetLemmatizer

In [2]:
NEO4J_BOLT_URL = "bolt://localhost:7687"
NEO4J_USERNAME = ""
NEO4J_PASSWORD = ""

In [3]:
selected_sentences = gutenberg.sents("bible-kjv.txt")

In [4]:
def get_english_stopwords():
    english_stopwords1 = "a's able about above according accordingly across actually after afterwards again against ain't all allow allows almost alone along already also although always am among amongst an and another any anybody anyhow anyone anything anyway anyways anywhere apart appear appreciate appropriate are aren't around as aside ask asking associated at available away awfully be became because become becomes becoming been before beforehand behind being believe below beside besides best better between beyond both brief but by c'mon c's came can can't cannot cant cause causes certain certainly changes clearly co com come comes concerning consequently consider considering contain containing contains corresponding could couldn't course currently definitely described despite did didn't different do does doesn't doing don't done down downwards during each edu eg eight either else elsewhere enough entirely especially et etc even ever every everybody everyone everything everywhere ex exactly example except far few fifth first five followed following follows for former formerly forth four from further furthermore get gets getting given gives go goes going gone got gotten greetings had hadn't happens hardly has hasn't have haven't having he he's hello help hence her here here's hereafter hereby herein hereupon hers herself hi him himself his hither hopefully how howbeit however i'd i'll i'm i've ie if ignored immediate in inasmuch inc indeed indicate indicated indicates inner insofar instead into inward is isn't it it'd it'll it's its itself just keep keeps kept know known knows last lately later latter latterly least less lest let let's like liked likely little look looking looks ltd mainly many may maybe me mean meanwhile merely might more moreover most mostly much must my myself name namely nd near nearly necessary need needs neither never nevertheless new next nine no nobody non none noone nor normally not nothing novel now nowhere obviously of off often oh ok okay old on once one ones only onto or other others otherwise ought our ours ourselves out outside over overall own particular particularly per perhaps placed please plus possible presumably probably provides que quite qv rather rd re really reasonably regarding regardless regards relatively respectively right said same saw say saying says second secondly see seeing seem seemed seeming seems seen self selves sensible sent serious seriously seven several shall she should shouldn't since six so some somebody somehow someone something sometime sometimes somewhat somewhere soon sorry specified specify specifying still sub such sup sure t's take taken tell tends th than thank thanks thanx that that's thats the their theirs them themselves then thence there there's thereafter thereby therefore therein theres thereupon these they they'd they'll they're they've think third this thorough thoroughly those though three through throughout thru thus to together too took toward towards tried tries truly try trying twice two un under unfortunately unless unlikely until unto up upon us use used useful uses using usually value various very via viz vs want wants was wasn't way we we'd we'll we're we've welcome well went were weren't what what's whatever when whence whenever where where's whereafter whereas whereby wherein whereupon wherever whether which while whither who who's whoever whole whom whose why will willing wish with within without won't wonder would wouldn't yes yet you you'd you'll you're you've your yours yourself yourselves zero".split(" ")

    # MySQL InnoDB full text search stopwords
    english_stopwords2 = "a,able,about,across,after,all,almost,also,am,among,an,and,any,are,as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,on,only,or,other,our,own,rather,said,say,says,she,should,since,so,some,than,that,the,their,them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,will,with,would,yet,you,your".split(",")
    return english_stopwords1 + english_stopwords2

In [5]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth(NEO4J_USERNAME, NEO4J_PASSWORD))
wordnet_lemmatizer = WordNetLemmatizer() 
puncuation = string.punctuation
english_stopwords = get_english_stopwords()
numbers = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

In [6]:
def create_constraint(tx):
    tx.run("CREATE CONSTRAINT ON (w: Word) ASSERT w.name IS UNIQUE")

In [7]:
def generate_sentence_query(tx, sentence):
    words = [word for word in sentence if word not in puncuation and word not in numbers]
    normalized_words = [wordnet_lemmatizer.lemmatize((word).lower()) for word in words]
    filtered_words = [word for word in normalized_words if word not in english_stopwords]
    tx.run("WITH {sentence} AS text "
           "UNWIND range(0, size(text)-2) AS i "
           "MERGE (w1: Word {name: text[i]}) ON CREATE SET w1.count = 1 ON MATCH SET w1.count = w1.count + 1 "
           "MERGE (w2: Word {name: text[i + 1]}) ON CREATE SET w2.count = 1 ON MATCH SET w2.count = w2.count + 1 "
           "MERGE (w1)-[r:NEXT]->(w2) ON CREATE SET r.count = 1 ON MATCH SET r.count = r.count + 1 ",
           sentence=filtered_words)

In [8]:
print("Started insertion")

number_of_finished_queries = 0

with driver.session() as session:
    session.write_transaction(create_constraint)

    for sentence in selected_sentences:
        session.write_transaction(generate_sentence_query, sentence)
        number_of_finished_queries = number_of_finished_queries + 1
        if (number_of_finished_queries % 500 == 0):
            print(number_of_finished_queries)
        
print("Ended insertion")

IndentationError: expected an indented block (<ipython-input-8-1d51b44709ea>, line 14)

In [9]:
english_stopwords

["a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 "c'mon",
 "c's",
 'came',
 'can',
 "can't",
 'cannot',
 'cant',
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 'co',
 'com',
 'come',
 'comes',
 'concernin